PySpark Assignment: Data Cleaning, Transformation, Analysis, and Prediction

Student:Angel Ivan Reyes Torresa ID: C0053883

Dataset Info: NBrewery Operations and Market Analysis Dat

2.6 GB

Total rows in dataset: 10,000,000

https://www.kaggle.com/datasets/ankurnapa/brewery-operations-and-market-analysis-dataset?select=brewery_data_complete_extended.csv

This dataset presents an extensive collection of data from a craft beer brewery, spanning from January 2020 to January 2024. It encapsulates a rich blend of brewing parameters, sales data, and quality assessments, providing a holistic view of the brewing process and its market implications.

Contain 20 columnsong others.

In [1]:
import pandas as pd
import seaborn as sns
from functools import reduce
import matplotlib.pyplot as plt
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from pyspark.ml.classification import LinearSVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, ParamGridBuilder
from pyspark.sql.types import FloatType, DoubleType, IntegerType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.sql.functions import col, count, when, isnan, to_date, year, month, dayofweek, dayofyear, avg

1 Data Loading and Initial Exploration

In [2]:
# Start a Spark session with optimized configurations for handling large datasets

spark = SparkSession.builder \
    .appName("BreweryDataAnalysis") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.sql.shuffle.partitions", "100") \
    .getOrCreate()

In [3]:
# Load the dataset with an appropriate number of partitions
file_path = "C:/Users/angel/00Angel/brewery_data.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [4]:
# Check the schema to see data types and structure
df.printSchema()

root
 |-- Batch_ID: integer (nullable = true)
 |-- Brew_Date: timestamp (nullable = true)
 |-- Beer_Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Fermentation_Time: integer (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- pH_Level: double (nullable = true)
 |-- Gravity: double (nullable = true)
 |-- Alcohol_Content: double (nullable = true)
 |-- Bitterness: integer (nullable = true)
 |-- Color: integer (nullable = true)
 |-- Ingredient_Ratio: string (nullable = true)
 |-- Volume_Produced: integer (nullable = true)
 |-- Total_Sales: double (nullable = true)
 |-- Quality_Score: double (nullable = true)
 |-- Brewhouse_Efficiency: double (nullable = true)
 |-- Loss_During_Brewing: double (nullable = true)
 |-- Loss_During_Fermentation: double (nullable = true)
 |-- Loss_During_Bottling_Kegging: double (nullable = true)



In [5]:
# Show the first 10 rows to inspect data quality and format
df.show(10, truncate=False)

+--------+-------------------+----------+-------+---------------+-----------------+------------------+------------------+------------------+-----------------+----------+-----+----------------+---------------+------------------+-----------------+--------------------+-------------------+------------------------+----------------------------+
|Batch_ID|Brew_Date          |Beer_Style|SKU    |Location       |Fermentation_Time|Temperature       |pH_Level          |Gravity           |Alcohol_Content  |Bitterness|Color|Ingredient_Ratio|Volume_Produced|Total_Sales       |Quality_Score    |Brewhouse_Efficiency|Loss_During_Brewing|Loss_During_Fermentation|Loss_During_Bottling_Kegging|
+--------+-------------------+----------+-------+---------------+-----------------+------------------+------------------+------------------+-----------------+----------+-----+----------------+---------------+------------------+-----------------+--------------------+-------------------+------------------------+-------

In [6]:
# Count the total number of rows in the dataset
row_count = df.count()
print(f"Total rows in dataset: {row_count}")

Total rows in dataset: 10000000


2 Data Cleaning and Transformation

2.1. Handling Missing Values

In [7]:
# Count missing values (nulls) for each column
missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

# Show the result
missing_values.show()

+--------+---------+----------+---+--------+-----------------+-----------+--------+-------+---------------+----------+-----+----------------+---------------+-----------+-------------+--------------------+-------------------+------------------------+----------------------------+
|Batch_ID|Brew_Date|Beer_Style|SKU|Location|Fermentation_Time|Temperature|pH_Level|Gravity|Alcohol_Content|Bitterness|Color|Ingredient_Ratio|Volume_Produced|Total_Sales|Quality_Score|Brewhouse_Efficiency|Loss_During_Brewing|Loss_During_Fermentation|Loss_During_Bottling_Kegging|
+--------+---------+----------+---+--------+-----------------+-----------+--------+-------+---------------+----------+-----+----------------+---------------+-----------+-------------+--------------------+-------------------+------------------------+----------------------------+
|       0|        0|         0|  0|       0|                0|          0|       0|      0|              0|         0|    0|               0|              0|      

In [8]:
df_cleaned = df.dropna()  # Removes rows with any null value

In [9]:
row_count = df_cleaned.count()
print(f"Total rows in dataset: {row_count}")

Total rows in dataset: 10000000


In [10]:
# Show the duplicate records (based on all columns)
duplicates_df = df.groupBy(df.columns).count().filter("count > 1")
duplicates_df.show()

+--------+---------+----------+---+--------+-----------------+-----------+--------+-------+---------------+----------+-----+----------------+---------------+-----------+-------------+--------------------+-------------------+------------------------+----------------------------+-----+
|Batch_ID|Brew_Date|Beer_Style|SKU|Location|Fermentation_Time|Temperature|pH_Level|Gravity|Alcohol_Content|Bitterness|Color|Ingredient_Ratio|Volume_Produced|Total_Sales|Quality_Score|Brewhouse_Efficiency|Loss_During_Brewing|Loss_During_Fermentation|Loss_During_Bottling_Kegging|count|
+--------+---------+----------+---+--------+-----------------+-----------+--------+-------+---------------+----------+-----+----------------+---------------+-----------+-------------+--------------------+-------------------+------------------------+----------------------------+-----+
+--------+---------+----------+---+--------+-----------------+-----------+--------+-------+---------------+----------+-----+----------------+----

2.3. Data Type Conversion

In [11]:
# Convert 'Brew_Date' column to date only (removes time part)
df = df.withColumn('Brew_Date', to_date(df['Brew_Date']))

# Show the result to confirm the changes
df.select('Brew_Date').show(5, truncate=False)

+----------+
|Brew_Date |
+----------+
|2020-01-01|
|2020-01-01|
|2020-01-01|
|2020-01-01|
|2020-01-01|
+----------+
only showing top 5 rows



In [12]:
df_cleaned = df.withColumn('Fermentation_Time', df['Fermentation_Time'].cast('integer'))
df_cleaned = df.withColumn('Brew_Date', df['Brew_Date'].cast('timestamp'))
df_cleaned = df.withColumn('Temperature', df['Temperature'].cast('double'))

2.4. Filtering Out Invalid Rows

In [13]:
# Filter out rows with invalid or impossible values
df_cleaned = df_cleaned.filter((df_cleaned['Volume_Produced'] >= 0) & 
                               (df_cleaned['Temperature'] >= 0) & (df_cleaned['Temperature'] <= 100) & 
                               (df_cleaned['pH_Level'] >= 0) & (df_cleaned['pH_Level'] <= 14))

1  Volume_Produced >= 0

Reason: The volume of beer produced (Volume_Produced) cannot be negative because it represents the actual quantity of beer brewed. Negative values would indicate an impossible or erroneous record. So, filtering out values less than zero is essential to ensure data validity.

2  Temperature >= 0 and Temperature <= 100

Reason: Minimum Temperature (0°C): The minimum possible temperature for water or beer is 0°C (freezing point of water). Temperatures below this could indicate data errors, such as negative values that don’t make sense in the context of brewing processes.

Maximum Temperature (100°C): 100°C is the boiling point of water at sea level. While brewing temperatures are typically below 100°C, extreme temperatures above 100°C would also be unusual. Therefore, this range is reasonable to filter out outliers or incorrect readings.
Brewing Temperature: Generally, brewing temperatures range between 60°C to 75°C for most beer styles. While this range is broader (0–100°C) to account for potential outliers, it still removes obviously invalid or extreme data points.

3 pH_Level >= 0 and pH_Level <= 14

Reason: Minimum pH (0): The pH scale ranges from 0 to 14, with 0 being extremely acidic and 14 being extremely alkaline. A pH level below 0 is physically impossible for a solution, so filtering values below 0 is necessary to remove errors.

Maximum pH (14): Similarly, a pH level above 14 is impossible in the context of beer brewing. Typical beer pH values range between 4.0 and 5.5, with some variation depending on the type of beer and fermentation conditions. Filtering values greater than 14 ensures that any erroneous or outlier values above the maximum pH scale are removed.

3 Data Analysis Using Spark SQL

3.1.1) Aggregation: Calculate summary statistics (e.g., mean, median, standard deviation) of the numerical columns.

In [14]:
df_final = df_cleaned
df_final.createOrReplaceTempView("brewing_data")

In [15]:

# Performing a SQL query to select all data from the brewing_data view
cleaned_data_query = "SELECT * FROM brewing_data"
final_df = spark.sql(cleaned_data_query)

# Showing the first few rows of the cleaned data
final_df.show(5)

+--------+----------+----------+----+------------+-----------------+------------------+------------------+------------------+-----------------+----------+-----+----------------+---------------+------------------+-----------------+--------------------+-------------------+------------------------+----------------------------+
|Batch_ID| Brew_Date|Beer_Style| SKU|    Location|Fermentation_Time|       Temperature|          pH_Level|           Gravity|  Alcohol_Content|Bitterness|Color|Ingredient_Ratio|Volume_Produced|       Total_Sales|    Quality_Score|Brewhouse_Efficiency|Loss_During_Brewing|Loss_During_Fermentation|Loss_During_Bottling_Kegging|
+--------+----------+----------+----+------------+-----------------+------------------+------------------+------------------+-----------------+----------+-----+----------------+---------------+------------------+-----------------+--------------------+-------------------+------------------------+----------------------------+
| 7870796|2020-01-01|W

In [16]:
# Identifying numeric columns in the cleaned data
numeric_columns = [field.name for field in final_df.schema.fields 
                   if isinstance(field.dataType, (FloatType, DoubleType, IntegerType))]

# Generating SQL queries for summary statistics (mean, median, standard deviation)
summary_query = f"""
SELECT 
    {', '.join([f'AVG({col}) AS avg_{col}' for col in numeric_columns])},
    {', '.join([f'PERCENTILE_APPROX({col}, 0.5) AS median_{col}' for col in numeric_columns])},
    {', '.join([f'STDDEV({col}) AS stddev_{col}' for col in numeric_columns])}
FROM brewing_data
"""

# Executing the SQL query to calculate summary statistics
summary_df = spark.sql(summary_query)

# Showing the summary statistics DataFrame
summary_df.show(truncate=False)

+------------+---------------------+------------------+-----------------+------------------+-------------------+--------------+----------+-------------------+------------------+-----------------+------------------------+-----------------------+----------------------------+--------------------------------+---------------+------------------------+------------------+-----------------+------------------+----------------------+-----------------+------------+----------------------+------------------+--------------------+---------------------------+--------------------------+-------------------------------+-----------------------------------+------------------+------------------------+------------------+-------------------+--------------------+----------------------+------------------+-----------------+----------------------+------------------+--------------------+---------------------------+--------------------------+-------------------------------+-----------------------------------+
|avg_B

3.1.2) Grouping and Filtering: Group data by specific categories and calculate aggregations for each group. 

Highest Total_Sales per Brew_Date and Beer_Style

In [17]:
# SQL query to find the highest Total_Sales per Brew_Date and Beer_Style, ordered by date
aggregation_query = """
SELECT 
    Brew_Date,
    Beer_Style,
    MAX(Total_Sales) AS Highest_Total_Sales
FROM brewing_data
GROUP BY Brew_Date, Beer_Style
ORDER BY Brew_Date DESC, Highest_Total_Sales DESC
LIMIT 10
"""

# Execute the query
result_df = spark.sql(aggregation_query)

# Show the results
result_df.show(truncate=False)

+----------+----------+-------------------+
|Brew_Date |Beer_Style|Highest_Total_Sales|
+----------+----------+-------------------+
|2023-12-31|Sour      |19999.957753273946 |
|2023-12-31|Porter    |19998.713582154644 |
|2023-12-31|Pilsner   |19998.166499284678 |
|2023-12-31|Stout     |19991.554562673293 |
|2023-12-31|Wheat Beer|19988.27588543136  |
|2023-12-31|Lager     |19980.77426642725  |
|2023-12-31|Ale       |19965.82561229111  |
|2023-12-31|IPA       |19956.92365618578  |
|2023-12-30|Ale       |19996.39620222792  |
|2023-12-30|IPA       |19992.75046454753  |
+----------+----------+-------------------+



Highest Quality_Score per Brew_Date and Beer_Style

In [18]:
# SQL query to find the highest Quality_Score per Brew_Date and Beer_Style, ordered by date
aggregation_query = """
SELECT 
    Brew_Date,
    Beer_Style,
    MAX(Quality_Score) AS Highest_Quality_Score
FROM brewing_data
GROUP BY Brew_Date, Beer_Style
ORDER BY Brew_Date DESC, Highest_Quality_Score DESC
LIMIT 10
"""

# Execute the query
result_df = spark.sql(aggregation_query)

# Show the results
result_df.show(truncate=False)

+----------+----------+---------------------+
|Brew_Date |Beer_Style|Highest_Quality_Score|
+----------+----------+---------------------+
|2023-12-31|Sour      |9.999161865781595    |
|2023-12-31|Pilsner   |9.998151783823628    |
|2023-12-31|Ale       |9.9972227757805      |
|2023-12-31|Porter    |9.99697217893943     |
|2023-12-31|IPA       |9.996293456793538    |
|2023-12-31|Wheat Beer|9.994572202670671    |
|2023-12-31|Stout     |9.990243632639132    |
|2023-12-31|Lager     |9.984752221429648    |
|2023-12-30|Ale       |9.999722934959543    |
|2023-12-30|IPA       |9.999592070600983    |
+----------+----------+---------------------+



Highest Volume_Produced per Brew_Date and Beer_Style

In [19]:
# SQL query to find the highest Volume_Produced per Brew_Date and Beer_Style, ordered by date
aggregation_query = """
SELECT 
    Brew_Date,
    Beer_Style,
    MAX(Volume_Produced) AS Highest_Volume_Produced
FROM brewing_data
GROUP BY Brew_Date, Beer_Style
ORDER BY Brew_Date DESC, Highest_Volume_Produced DESC
LIMIT 10
"""

# Execute the query
result_df = spark.sql(aggregation_query)

# Show the results
result_df.show(truncate=False)

+----------+----------+-----------------------+
|Brew_Date |Beer_Style|Highest_Volume_Produced|
+----------+----------+-----------------------+
|2023-12-31|Ale       |4999                   |
|2023-12-31|Pilsner   |4999                   |
|2023-12-31|Stout     |4998                   |
|2023-12-31|IPA       |4996                   |
|2023-12-31|Sour      |4996                   |
|2023-12-31|Lager     |4993                   |
|2023-12-31|Wheat Beer|4990                   |
|2023-12-31|Porter    |4986                   |
|2023-12-30|Wheat Beer|4999                   |
|2023-12-30|Lager     |4997                   |
+----------+----------+-----------------------+



3.1.3) Joins: If applicable, perform a join between two tables

Are not applicable for our dataset.

3.1.4) Time-based analysis: If your dataset contains a timestamp column, analyze trends over time.

SQL query for number of batches and total sales per Brew_Date

In [20]:
# SQL query to find the number of batches and total sales per Brew_Date
time_based_query = """
SELECT 
    Brew_Date,
    COUNT(Batch_ID) AS Number_of_Batches,
    SUM(Total_Sales) AS Total_Sales_Per_Day
FROM brewing_data
GROUP BY Brew_Date
ORDER BY Brew_Date DESC
LIMIT 10
"""

# Execute the query
result_time_based_df = spark.sql(time_based_query)

# Show the results
result_time_based_df.show(truncate=False)


+----------+-----------------+-------------------+
|Brew_Date |Number_of_Batches|Total_Sales_Per_Day|
+----------+-----------------+-------------------+
|2023-12-31|6951             |7.402164022621739E7|
|2023-12-30|6875             |7.277619057643037E7|
|2023-12-29|6801             |7.101619643852998E7|
|2023-12-28|6853             |7.161239863457432E7|
|2023-12-27|6823             |7.105070785238072E7|
|2023-12-26|6778             |7.173696041901506E7|
|2023-12-25|6975             |7.328681953301111E7|
|2023-12-24|6945             |7.398418149452E7   |
|2023-12-23|6809             |7.171651056100419E7|
|2023-12-22|6787             |7.072098217357387E7|
+----------+-----------------+-------------------+



SQL Query for Average Quality Score and Alcohol Content per Brew

In [21]:
# SQL query to find the average Quality Score and Alcohol Content per Brew_Date
average_metrics_query = """
SELECT 
    Brew_Date,
    AVG(Quality_Score) AS Average_Quality_Score,
    AVG(Alcohol_Content) AS Average_Alcohol_Content
FROM brewing_data
GROUP BY Brew_Date
ORDER BY Brew_Date DESC
LIMIT 10
"""

# Execute the query
result_average_metrics_df = spark.sql(average_metrics_query)

# Show the results
result_average_metrics_df.show(truncate=False)


+----------+---------------------+-----------------------+
|Brew_Date |Average_Quality_Score|Average_Alcohol_Content|
+----------+---------------------+-----------------------+
|2023-12-31|7.972432599968671    |5.247159001244369      |
|2023-12-30|7.972838103070529    |5.246944176486663      |
|2023-12-29|8.000243096635971    |5.2470258847132065     |
|2023-12-28|8.01174732752383     |5.2542033652894995     |
|2023-12-27|7.994104801119074    |5.239622772825028      |
|2023-12-26|7.998919681970579    |5.245127095144804      |
|2023-12-25|8.01056465475819     |5.244991263699129      |
|2023-12-24|7.993671636436315    |5.253078226738263      |
|2023-12-23|7.992270025832132    |5.248499193071808      |
|2023-12-22|7.99786223144857     |5.257680459953336      |
+----------+---------------------+-----------------------+



SQL Query for Beer Style Count per Brew Date

In [22]:
# SQL query to count the number of each Beer Style produced per Brew_Date
beer_style_count_query = """
SELECT 
    Brew_Date,
    Beer_Style,
    COUNT(*) AS Beer_Style_Count
FROM brewing_data
GROUP BY Brew_Date, Beer_Style
ORDER BY Brew_Date DESC, Beer_Style_Count DESC
LIMIT 10
"""

# Execute the query
result_beer_style_count_df = spark.sql(beer_style_count_query)

# Show the results
result_beer_style_count_df.show(truncate=False)

+----------+----------+----------------+
|Brew_Date |Beer_Style|Beer_Style_Count|
+----------+----------+----------------+
|2023-12-31|Lager     |892             |
|2023-12-31|Ale       |879             |
|2023-12-31|Wheat Beer|877             |
|2023-12-31|IPA       |876             |
|2023-12-31|Stout     |876             |
|2023-12-31|Pilsner   |864             |
|2023-12-31|Sour      |853             |
|2023-12-31|Porter    |834             |
|2023-12-30|Sour      |905             |
|2023-12-30|Stout     |901             |
+----------+----------+----------------+



Insights:

1 Highest Total Sales
   
Top Performers: On 2023-12-31, Sour beer led in total sales, followed closely by Porter, Pilsner, and Stout.
Beer Style Performance: Most beer styles show high total sales on 2023-12-31, indicating that this was likely a peak day for sales overall.
Sales Trend: The sales on 2023-12-30 were slightly lower, with Ale and IPA performing well, although sales for other styles were still significant.

2 Highest Quality Score

Top Performers: Sour beer had the highest quality score on 2023-12-31, followed by Pilsner and Ale. This suggests that consumers rated these beer styles very highly.
Quality Consistency: Quality scores across all beer styles are consistently high, particularly on 2023-12-30, where the Ale beer style achieved almost a perfect score.

3 Highest Volume Produced

Volume Consistency: Ale and Pilsner beer styles led in volume produced on 2023-12-31, both reaching 4999 units, indicating significant production for these styles.
Production Trends: Stout and IPA also had high production numbers, but the volume produced slightly dropped for some styles like Porter.

4 Number of Batches and Sales per Day

Batches and Sales Volume: 2023-12-31 recorded the highest number of batches (6951) and highest sales per day (around 74 million), suggesting that it was a high-volume production day.
Sales Correlation: The total sales per day shows fluctuations but remains high on 2023-12-31. The data indicates a very active production and sales environment, possibly around the holiday season.

5 Average Quality Score and Alcohol Content

Consistency in Quality Score: The average quality scores remain high across all the days, with a slight dip on 2023-12-31.
Alcohol Content: Average alcohol content remains relatively stable, with 2023-12-31 having an average of about 5.25%. This suggests no major variations in the types of beer produced during this period.

6 Beer Style Count

Lager and Ale Dominance: Lager and Ale were the most produced and sold beers on 2023-12-31, with Lager topping the list at 892 batches.
Beer Style Spread: The number of batches for Sour and Porter were slightly lower compared to others, indicating less production or popularity during this period.

Key Observations:

Sales and Production Peaks: The peak sales figures on 2023-12-31 align with increased production and higher quality scores for various beer styles, especially Sour and Pilsner.

Quality and Volume Correlation: Beer styles with high total sales on 2023-12-31 (like Sour and Pilsner) also had some of the highest quality scores, which indicates a strong positive correlation between quality and sales.

Beer Preferences: Ale and IPA are the more commonly produced beer styles, while Sour and Porter might have smaller but more targeted batches, with high-quality ratings.

4 Machine Learning Model (Regression/Classification)

4.1 Choose the appropriate ML problem based on the dataset:

Justification for Regression Model Selection
I'm selecting a regression model to predict Total Sales because:

Nature of the Target Variable (Total Sales):

Total Sales is a continuous numeric value, making it a natural fit for regression. The goal is to predict a specific value that can take on any numeric value within a range, which is a hallmark of regression problems.

The primary objective is to predict Total Sales based on various influencing factors such as Beer Style, Number of Batches, and Average Quality Score. This prediction can provide valuable insights for forecasting, budget allocation, and sales strategy, making it a key decision-making tool.
Using regression allows us to generate continuous predictions for total sales, which can be leveraged to adjust strategies in real-time based on predicted values.

Predictive Power:

Since Total Sales is directly influenced by multiple continuous features (e.g., Beer Style Count, Average Alcohol Content, Brew_Date, etc.), regression models are well-suited to capture these relationships and provide accurate numerical predictions. This can help in resource planning, inventory management, and understanding demand trends.
Given this, the regression approach is optimal for our dataset, where we want to predict the Total Sales based on a variety of features.

Model: Random Forest Regressor

Random Forest Regressor for the following reasons:

Handles Non-linearity: The relationships between Total Sales and features like Beer Style Count, Average Quality Score, and Number of Batches may be complex and non-linear. Random Forest can capture these complex relationships, unlike linear models.

Robustness: It is less prone to overfitting compared to a single decision tree, as it uses an ensemble of trees. This makes it a more robust choice when dealing with real-world, noisy data.

Feature Importance: Random Forest provides insights into feature importance, which can help identify which features contribute most to predicting Total Sales, aiding in business decisions.

Accuracy: It generally provides better predictive accuracy compared to simpler models like linear regression, especially for more complex problems with multiple interacting features.

In [23]:
print(type(df_cleaned))

<class 'pyspark.sql.dataframe.DataFrame'>


In [24]:
df_cleaned.show(5)

+--------+----------+----------+----+------------+-----------------+------------------+------------------+------------------+-----------------+----------+-----+----------------+---------------+------------------+-----------------+--------------------+-------------------+------------------------+----------------------------+
|Batch_ID| Brew_Date|Beer_Style| SKU|    Location|Fermentation_Time|       Temperature|          pH_Level|           Gravity|  Alcohol_Content|Bitterness|Color|Ingredient_Ratio|Volume_Produced|       Total_Sales|    Quality_Score|Brewhouse_Efficiency|Loss_During_Brewing|Loss_During_Fermentation|Loss_During_Bottling_Kegging|
+--------+----------+----------+----+------------+-----------------+------------------+------------------+------------------+-----------------+----------+-----+----------------+---------------+------------------+-----------------+--------------------+-------------------+------------------------+----------------------------+
| 7870796|2020-01-01|W

Feature Engineering

In [25]:
df_cleaned.columns

['Batch_ID',
 'Brew_Date',
 'Beer_Style',
 'SKU',
 'Location',
 'Fermentation_Time',
 'Temperature',
 'pH_Level',
 'Gravity',
 'Alcohol_Content',
 'Bitterness',
 'Color',
 'Ingredient_Ratio',
 'Volume_Produced',
 'Total_Sales',
 'Quality_Score',
 'Brewhouse_Efficiency',
 'Loss_During_Brewing',
 'Loss_During_Fermentation',
 'Loss_During_Bottling_Kegging']

In [26]:
# Filter out non-numeric columns
numeric_cols = [col for col, dtype in df_cleaned.dtypes if dtype in ('int', 'double')]

# Select only the numeric columns
numeric_df = df_cleaned.select(*numeric_cols)

# Calculate correlation with 'Total_Sales' for each numeric column
for col in numeric_df.columns:
    if col != 'Total_Sales':  # Skip correlation with itself
        corr_val = numeric_df.stat.corr(col, 'Total_Sales')
        print(f"Correlation between {col} and Total_Sales: {corr_val}")

Correlation between Batch_ID and Total_Sales: -0.00015943984849964196
Correlation between Fermentation_Time and Total_Sales: 0.0001702529220165709
Correlation between Temperature and Total_Sales: 4.812389524730911e-05
Correlation between pH_Level and Total_Sales: -4.858993765998827e-05
Correlation between Gravity and Total_Sales: 0.000514309198615154
Correlation between Alcohol_Content and Total_Sales: 0.0005225099074256979
Correlation between Bitterness and Total_Sales: -0.00015433550969259514
Correlation between Color and Total_Sales: 0.00014694286695053862
Correlation between Volume_Produced and Total_Sales: 4.539233729138603e-05
Correlation between Quality_Score and Total_Sales: 0.000516940642588387
Correlation between Brewhouse_Efficiency and Total_Sales: 0.00018469634422112627
Correlation between Loss_During_Brewing and Total_Sales: 0.00016438722970381857
Correlation between Loss_During_Fermentation and Total_Sales: -0.0003096972210355463
Correlation between Loss_During_Bottling_

In [27]:
from pyspark.sql.functions import col

# Calculate Brew_Year_Efficiency
df_cleaned = df_cleaned.withColumn(
    "Brew_Year_Efficiency", col("Volume_Produced") / col("Brewhouse_Efficiency")
)

In [28]:
# Calculate Number_of_Batches as the count of distinct Batch_ID per SKU
batch_counts = df_cleaned.groupBy("SKU").agg({"Batch_ID": "count"}).withColumnRenamed("count(Batch_ID)", "Number_of_Batches")

# Join the batch counts back to the original DataFrame
df_cleaned = df_cleaned.join(batch_counts, on="SKU", how="left")

In [29]:
# Calculate Volume_per_Batch
df_cleaned = df_cleaned.withColumn(
    "Volume_per_Batch", col("Volume_Produced") / col("Number_of_Batches")
)

In [30]:
# Extract the year from the Brew_Date column and create a new column 'Brew_Year'
df_cleaned = df_cleaned.withColumn("Brew_Year", year("Brew_Date"))

In [31]:
df_cleaned.select("Brew_Year", "Volume_Produced", "Brewhouse_Efficiency", "Brew_Year_Efficiency", "Number_of_Batches", "Volume_per_Batch").show(5)


+---------+---------------+--------------------+--------------------+-----------------+--------------------+
|Brew_Year|Volume_Produced|Brewhouse_Efficiency|Brew_Year_Efficiency|Number_of_Batches|    Volume_per_Batch|
+---------+---------------+--------------------+--------------------+-----------------+--------------------+
|     2020|           4449|   88.62583302052388|   50.19981024008773|          2498777|0.001780471006416...|
|     2020|           4666|   89.19588216376087|   52.31183196813244|          2498753|0.001867331424914...|
|     2020|            832|    72.4809153900275|  11.478883724397237|          2498753|3.329660834824410...|
|     2020|           2115|   86.32214396020584|  24.501245022076915|          2498753|8.464221953910611E-4|
|     2020|           3173|   83.09494037181545|   38.18523710110554|          2498753|0.001269833392896...|
+---------+---------------+--------------------+--------------------+-----------------+--------------------+
only showing top 5 

In [32]:
# Recalculate the correlation matrix including the new columns
columns_to_check = ["Brew_Year", "Volume_Produced", "Brewhouse_Efficiency", "Brew_Year_Efficiency", 
                    "Number_of_Batches", "Volume_per_Batch", "Total_Sales"]

# Loop over the columns and calculate the correlation with Total_Sales
for col_name in columns_to_check:
    corr_val = df_cleaned.stat.corr(col_name, 'Total_Sales')
    print(f"Correlation between {col_name} and Total_Sales: {corr_val}")

Correlation between Brew_Year and Total_Sales: -0.0006929445859795882
Correlation between Volume_Produced and Total_Sales: 4.539233729138603e-05
Correlation between Brewhouse_Efficiency and Total_Sales: 0.00018469634422112627
Correlation between Brew_Year_Efficiency and Total_Sales: 4.126405403179541e-06
Correlation between Number_of_Batches and Total_Sales: 8.185164315756284e-05
Correlation between Volume_per_Batch and Total_Sales: 4.522094621693578e-05
Correlation between Total_Sales and Total_Sales: 1.0


Identify Key Predictors: Understanding which features have a high correlation with Total_Sales helps us identify which variables might be more influential in predicting Total_Sales. Highly correlated features are usually strong candidates for inclusion in a predictive model.

Feature Selection: If certain features are highly correlated with the target, they may be more valuable for model training, while features with little to no correlation might be less useful. This step helps us prioritize which columns to keep as predictors and which to potentially exclude.

New Features Assessment: Since some new columns (like Brew_Year_Efficiency and Volume_per_Batch) were recently added, this process allows us to assess whether these new features provide additional explanatory power for predicting Total_Sales.

Insight into Relationships: The correlation matrix gives insights into relationships between variables in the data. Positive correlations suggest that as one variable increases, so does Total_Sales, while negative correlations suggest the opposite.

By recalculating the correlation matrix with the new columns, we refine our understanding of the data and its key drivers, which informs subsequent model-building steps.

In [33]:
# Create a new DataFrame with the selected columns
df_selected = df_cleaned.select("Brew_Year", "Volume_Produced", "Brewhouse_Efficiency", "Number_of_Batches", "Total_Sales")

# Show the first few rows to verify
df_selected.show(5)

+---------+---------------+--------------------+-----------------+------------------+
|Brew_Year|Volume_Produced|Brewhouse_Efficiency|Number_of_Batches|       Total_Sales|
+---------+---------------+--------------------+-----------------+------------------+
|     2020|           4449|   88.62583302052388|          2498777|5515.0774647529615|
|     2020|           4666|   89.19588216376087|          2498753|2664.7593448382822|
|     2020|            832|    72.4809153900275|          2498753| 9758.801062471319|
|     2020|           2115|   86.32214396020584|          2498753|11721.087016274963|
|     2020|           3173|   83.09494037181545|          2498753|12050.177463190277|
+---------+---------------+--------------------+-----------------+------------------+
only showing top 5 rows



Feature Selection:

I first performed a correlation analysis between the target variable Total_Sales and other columns in the dataset to identify potential relationships.
Based on the correlation results, I identified the columns with the strongest relationships with Total_Sales. The columns that were kept for further analysis were:

Volume_Produced: Represents the total volume of beer produced, which likely influences total sales.

Brewhouse_Efficiency: Indicates how efficiently the brewhouse operates, which can impact production output and sales.

Number_of_Batches: Represents the number of production batches, which may correlate with the total volume produced and ultimately affect sales.

Columns like Brew_Year_Efficiency, Brew_Year, and Volume_per_Batch were not retained due to their weaker correlations with Total_Sales.

New DataFrame Creation:

After feature selection, I created a new DataFrame (df_selected) containing only the relevant columns: Volume_Produced, Brewhouse_Efficiency, Number_of_Batches, and Total_Sales.

This new DataFrame serves as the clean, simplified dataset that can be used for further modeling and analysis.

The goal of this step was to reduce dimensionality and retain only the most influential features for predicting Total_Sales.

By removing weaker or redundant features, we improve the model's performance and efficiency, ensuring that the machine learning model can focus on the key factors driving sales.

In [34]:
from pyspark.ml.feature import StringIndexer

# Step 1: Use StringIndexer to convert Brew_Year to a numeric index
indexer = StringIndexer(inputCol="Brew_Year", outputCol="Brew_Year_Index")
df_selected = indexer.fit(df_selected).transform(df_selected)

# Step 2: Drop the original Brew_Year column
df_selected = df_selected.drop("Brew_Year")

+---------------+--------------------+-----------------+------------------+---------------+
|Volume_Produced|Brewhouse_Efficiency|Number_of_Batches|       Total_Sales|Brew_Year_Index|
+---------------+--------------------+-----------------+------------------+---------------+
|            593|   85.73496269913532|          2500873|14362.653665879505|            0.0|
|           4894|   85.26361099177572|          2500873|11852.097970324789|            0.0|
|           4486|   89.49078286225225|          2500873|1665.1818593163712|            0.0|
|           3809|   88.37087230231108|          2500873| 5925.879914754564|            0.0|
|           4727|   70.66652472059178|          2500873|19228.362127803455|            0.0|
+---------------+--------------------+-----------------+------------------+---------------+
only showing top 5 rows



We applied a StringIndexer to the Brew_Year column, creating a new column, Brew_Year_Index, with numerical values instead of categorical text values.

Why We Did It:

Machine Learning Compatibility: Most machine learning algorithms cannot directly process categorical string data. They require features to be in numerical form. StringIndexer converts each unique text category into a unique numeric index.
Encoding Categories: StringIndexer assigns a numerical index to each unique value in Brew_Year. For example, if Brew_Year contains values like "2020", "2021", and "2022", each one will be mapped to a distinct number (like 0, 1, 2, etc.). This maintains the categorical information in a machine-readable form without implying any specific order among years.

Facilitates Vectorization: This numeric index allows us to later combine it into a single feature vector, which we’ll use for model training.

In [39]:
# Show the resulting DataFrame with the indexed Brew_Year
df_selected.show(5)

+---------------+--------------------+-----------------+------------------+---------------+
|Volume_Produced|Brewhouse_Efficiency|Number_of_Batches|       Total_Sales|Brew_Year_Index|
+---------------+--------------------+-----------------+------------------+---------------+
|            593|   85.73496269913532|          2500873|14362.653665879505|            0.0|
|           4894|   85.26361099177572|          2500873|11852.097970324789|            0.0|
|           4486|   89.49078286225225|          2500873|1665.1818593163712|            0.0|
|           3809|   88.37087230231108|          2500873| 5925.879914754564|            0.0|
|           4727|   70.66652472059178|          2500873|19228.362127803455|            0.0|
+---------------+--------------------+-----------------+------------------+---------------+
only showing top 5 rows



In [38]:
df_selected.select("Brew_Year_Index").distinct().show()

+---------------+
|Brew_Year_Index|
+---------------+
|            0.0|
|            3.0|
|            2.0|
|            1.0|
+---------------+



VectorAssembler: Combine all features into a single vector.

In [42]:
from pyspark.ml.feature import VectorAssembler

# Step 1: Define the feature columns
feature_columns = ['Volume_Produced', 'Brewhouse_Efficiency', 'Number_of_Batches', 'Total_Sales', 'Brew_Year_Index']

# Step 2: Create the VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Step 3: Transform the DataFrame
df_transformed = assembler.transform(df_selected)

# Step 4: Show the result (including the 'features' column)
df_transformed.select("features").show(5)

+--------------------+
|            features|
+--------------------+
|[593.0,85.7349626...|
|[4894.0,85.263610...|
|[4486.0,89.490782...|
|[3809.0,88.370872...|
|[4727.0,70.666524...|
+--------------------+
only showing top 5 rows



The VectorAssembler is used in machine learning workflows to combine multiple individual feature columns into a single vector column. This is necessary because most machine learning algorithms in Spark (and other ML frameworks) expect input data in a specific format—typically as a single column of vectors, where each vector represents a row of feature values.

Reasons for using VectorAssembler:
Required Input Format: Machine learning algorithms in Spark expect a single column containing a vector for the features, rather than multiple columns. The VectorAssembler consolidates all the feature columns (e.g., numerical, categorical) into a single vector column.

Simplicity and Efficiency: It simplifies data manipulation, especially when working with many features. Rather than manually combining each feature, VectorAssembler automatically handles this process, saving time and reducing errors.

Facilitates Model Training: Once the features are assembled into a vector, they can be directly fed into Spark’s machine learning models like regression, classification, and clustering. Without this step, these models would not be able to accept the input data.

In essence, the VectorAssembler prepares the data for use in machine learning algorithms by transforming it into the required vector format, making it a crucial step in most Spark ML workflows.

Model evaluation: Evaluate the model using metrics such as RMSE for regression or
accuracy/F1-score for classification

In [43]:
# 'Total_Sales' is the target column

# Step 1: Define the target and feature columns
target_column = 'Total_Sales'

# Step 2: Split the data into training and test sets
train_df, test_df = df_transformed.randomSplit([0.8, 0.2], seed=42)

In [44]:
# Step 2: Initialize the Random Forest Regressor
rf = RandomForestRegressor(featuresCol="features", labelCol=target_column)

# Step 3: Train the model on the training data
rf_model = rf.fit(train_df)

In [45]:
# Step 4: Make predictions on the test set
predictions = rf_model.transform(test_df)

# Step 5: Show the predictions (optional)
predictions.select("features", "prediction", target_column).show(5)

+--------------------+------------------+------------------+
|            features|        prediction|       Total_Sales|
+--------------------+------------------+------------------+
|[500.0,70.0686782...| 4357.217992752436|3168.6924853628825|
|[500.0,70.2252333...|16541.635464101542|15666.325745637327|
|[500.0,70.2668015...|  12672.3196317226|11095.155580607869|
|[500.0,70.4654092...| 4184.369720752677| 4418.922397793351|
|[500.0,70.6384091...| 12715.37223075588|12620.931290672395|
+--------------------+------------------+------------------+
only showing top 5 rows



While there are some variances, these differences are relatively small in the context of the sales figures, supporting the strong performance metrics (high R² and low RMSE).

Overall Insight:

The model is accurately predicting Total_Sales with a high degree of reliability, evidenced by the close alignment between predictions and actual values across different feature combinations. Minor differences suggest room for marginal improvements, but overall, the model is performing well in approximating Total_Sales.

In [46]:
# Step 6: Evaluate the model using RMSE and R2
evaluator_rmse = RegressionEvaluator(labelCol=target_column, predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol=target_column, predictionCol="prediction", metricName="r2")

# Compute RMSE and R2
rmse = evaluator_rmse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)

print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R2: {r2}")

Root Mean Squared Error (RMSE): 1250.358281977003
R2: 0.94806040370784


RMSE represents the square root of the average squared differences between predicted and actual values. In simple terms, it gives you the average error magnitude in the units of your target variable (in this case, Total_Sales).

Generally, the lower the RMSE, the better the model, but whether this value is acceptable depends on the scale of your target variable. For example, if Total_Sales typically ranges in the thousands or millions, an RMSE of around 1250 may be reasonable. If the values are much smaller, the RMSE might be relatively high, suggesting room for improvement.

R²: 0.9481

The R² (Coefficient of Determination) indicates how well your model can explain the variance in the target variable, Total_Sales. In this case, 0.9481 means that the model explains about 94.81% of the variance in Total_Sales.

An R² value close to 1 suggests a strong model fit. A value of 0.9481 is very good, indicating that the model does a great job of predicting sales, with only a small portion of the variance left unexplained.


Good Model Fit: With an R² of 0.9481, your model explains a high percentage of the variance in Total_Sales, which suggests that your model is quite effective.

Moderate Error (RMSE): The RMSE value of 1250.36 suggests that while your model's predictions are quite accurate, there is still some room for improvement

Model Performance: The combination of a low RMSE and high R² suggests that your Random Forest model is performing well, with good predictive power, even though there are small errors. It's a solid model to work with, and improvements could make it even better.

Conclusion:
The model is providing solid predictions with an R² of around 94.81%, indicating strong performance. 

In [47]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Step 7: Hyperparameter tuning (optional)
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [10, 20])
             .addGrid(rf.maxDepth, [5, 10])
             .build())

crossval = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator_rmse, numFolds=3)

# Step 8: Train the model with hyperparameter tuning
cv_model = crossval.fit(train_df)
best_model = cv_model.bestModel

# Step 9: Make predictions with the best model
best_predictions = best_model.transform(test_df)

In [48]:
# Initialize evaluator with target metric
evaluator_rmse = RegressionEvaluator(labelCol="Total_Sales", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="Total_Sales", predictionCol="prediction", metricName="r2")

# Compute RMSE and R²
rmse = evaluator_rmse.evaluate(best_predictions)
r2 = evaluator_r2.evaluate(best_predictions)

print(f"Test RMSE: {rmse}")
print(f"Test R²: {r2}")

Test RMSE: 411.9342990211842
Test R²: 0.9943625133289884


Root Mean Squared Error (RMSE): 411.93

The RMSE value represents the average error in terms of units of the target variable (Total_Sales) across the test set. An RMSE of ~411 suggests that, on average, the predictions are off by around 411 units. Given the scale of your Total_Sales values, this indicates relatively low error and good accuracy.

R-squared (R²): 0.994

The R² value, which is 0.994, indicates that your model explains about 99.4% of the variance in Total_Sales. This high R² suggests that the model captures most of the variability in the data, leaving very little unexplained variance. It’s an excellent result and shows that the model has a strong fit to the data.